In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from mysql.connector import (connection)
from collections import defaultdict
cnx = connection.MySQLConnection(user='username', password='pass',
                                 host='127.0.0.1',
                                 database='database')


In [2]:
keeper = []
truecar = defaultdict(list)
vehicle = input('what car are u looking for?(brand and model): ').split(' ')

try:
    res = requests.get('https://www.truecar.com/used-cars-for-sale/listings/%s/%s/' % (vehicle[0],vehicle[1],))

    if res.status_code==403:
        raise Exception('\nSorry, the website you are trying to send REST API request is forbbiden 403 Error.')


except ConnectTimeoutError as e:
    print('Connection timed out!.')

except Exception as e:
    print("Sorry,something wnet wrong ...!" , e)


else:
    if res.status_code==200:
        print('API request successfully have been sent.')
    
    for i in range (1,10):
        
        res = requests.get('https://www.truecar.com/used-cars-for-sale/listings/%s/%s/?page=%i' % (vehicle[0],vehicle[1],i))
        soup = BeautifulSoup(res.text,'html.parser')
        cars = soup.find_all('div',attrs = {'data-test':"cardContent", 'class':"card-content order-3 vehicle-card-body"})
        
        for car in cars:
            want = re.findall(r'\w*?(\d+ \w+ \w+) .* list price(\$\d+\,\d{3})*(\$\d+\,\d{3})(\d+\,*\d*)',car.text)
            if want==[]:
                continue
            final = []
            for i in want:
                for j in i:
                    if j!='':
                        final.append(j)
            if len(final)==4:
                final.remove(final[1])    
            
            all = final[0].split(' ')
            year = all[0]
            year = int(year)
            brand = all[1]
            model = all[2]
            price = final[1]
            mile = final[2]
            if final in keeper:
                continue
            else:
                keeper.extend(final)
        
            cursor = cnx.cursor()
            cursor.execute('INSERT INTO cars VALUES (\'%s\',\'%s\' , \'%i\' , \'%s\' , \'%s\') '  % ((brand,model,year,price,mile)))
            cnx.commit()        
            #print('%s %s %i is %s and mileage is %s'%(brand,model,year,price,mile))


    cursor = cnx.cursor()    
    query = 'SELECT * FROM cars WHERE brand = brand or model = model;'    
    cursor.execute(query)

    for brand,model,year,price,mile in cursor:
        mile = mile.split(',')
        mile = ''.join(mile)
        mile = int(mile)
        price = price.split('$')
        price = price[1].split(',')
        price = ''.join(price)
        price = int(price)
        truecar["brand"].append(brand)
        truecar["model"].append(model)
        truecar["year"].append(year)
        truecar["price$"].append(price)
        truecar["mileage"].append(mile)

    
    removal = 'TRUNCATE TABLE cars;' 
# for excecution:
    cursor.execute(removal)
finally:    
    cnx.close() 
    print('revale')

df = pd.DataFrame(truecar)
print(df)

what car are u looking for?(brand and model):  lamborghini huracan


API request successfully have been sent.
revale
           brand                  model  year  price$  mileage
0         Nissan                VersaSV  2021   11995    63150
1           Ford            ExplorerXLT  2020   19900    94741
2         Nissan                VersaSV  2021   14598    64097
3      Chevrolet              EquinoxLT  2019   14367    77349
4      Chevrolet              EquinoxLT  2020   16698    78322
..           ...                    ...   ...     ...      ...
587  Lamborghini           HuracanCoupe  2019  239999     5247
588  Lamborghini              HuracanLP  2015  197991    21854
589  Lamborghini           HuracanCoupe  2020  239988    19596
590  Lamborghini      HuracanCoupeGreat  2023  339500     2020
591  Lamborghini  HuracanCoupeExcellent  2023  314991     3254

[592 rows x 5 columns]


In [3]:
filt = (df['brand']=='BMW') & (df['year'] > 2020)
new_df = df.loc[filt]
new_df

,brand,model,year,price$,mileage
40,BMW,s,2021,25900,62813
58,BMW,s,2021,25900,62813
66,BMW,120,2021,30997,31700
70,BMW,120,2021,27998,17370
72,BMW,s,2021,34174,6731
114,BMW,i3s,2021,25900,62813
142,BMW,i3120,2021,30997,31700
156,BMW,i3120,2021,27998,17370
165,BMW,i3s,2021,34174,6731
175,BMW,i3s,2021,27995,26753


In [4]:
df['price$'] = df['price$'].apply(lambda x: ('%i$' % x))

In [5]:
df.columns = df.columns.str.capitalize()

In [6]:
df

,Brand,Model,Year,Price$,Mileage
0,Nissan,VersaSV,2021,11995$,63150
1,Ford,ExplorerXLT,2020,19900$,94741
2,Nissan,VersaSV,2021,14598$,64097
3,Chevrolet,EquinoxLT,2019,14367$,77349
4,Chevrolet,EquinoxLT,2020,16698$,78322
...,...,...,...,...,...
587,Lamborghini,HuracanCoupe,2019,239999$,5247
588,Lamborghini,HuracanLP,2015,197991$,21854
589,Lamborghini,HuracanCoupe,2020,239988$,19596
590,Lamborghini,HuracanCoupeGreat,2023,339500$,2020


In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.reset_index()

,index,Brand,Model,Year,Price$,Mileage
0,0,Nissan,VersaSV,2021,11995$,63150
1,1,Ford,ExplorerXLT,2020,19900$,94741
2,2,Nissan,VersaSV,2021,14598$,64097
3,3,Chevrolet,EquinoxLT,2019,14367$,77349
4,4,Chevrolet,EquinoxLT,2020,16698$,78322
...,...,...,...,...,...,...
255,293,Lamborghini,HuracanSpyder,2017,229991$,30452
256,294,Lamborghini,HuracanLP,2016,199998$,33844
257,295,Lamborghini,HuracanCoupe,2017,229998$,12844
258,296,Lamborghini,HuracanPerformante,2018,359998$,8776


In [9]:
filt = df['Brand'] == 'Lamborghini'
lambo = df.loc[filt]

In [10]:
lambo.reset_index()

,index,Brand,Model,Year,Price$,Mileage
0,178,Lamborghini,HuracanSTO,2023,375500$,1693
1,179,Lamborghini,HuracanSpyder,2017,249756$,1428
2,180,Lamborghini,HuracanCoupe,2020,279596$,5512
3,181,Lamborghini,HuracanSTO,2022,389900$,172
4,182,Lamborghini,HuracanSpyder,2018,225500$,9196
...,...,...,...,...,...,...
115,293,Lamborghini,HuracanSpyder,2017,229991$,30452
116,294,Lamborghini,HuracanLP,2016,199998$,33844
117,295,Lamborghini,HuracanCoupe,2017,229998$,12844
118,296,Lamborghini,HuracanPerformante,2018,359998$,8776
